In [1]:
! pip install mlxtend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.9 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
from collections import defaultdict
import scipy.stats as stat
import importlib


import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler


from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

In [2]:
df_train=pd.read_csv("house_price_train.csv")

In [3]:
df_train.drop("Id",axis=1,inplace=True) ## reduntatnt
output_col=df_train['SalePrice']
df_train.drop('SalePrice',axis=1,inplace=True)


In [4]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


In [5]:
df_train.describe()


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000


In [6]:
df_train.isnull().sum()

MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Street             0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
Length: 79, dtype: int64

## PREPROCESSING


In [7]:
# df_train.columns[df_train.isnull().any()]

In [8]:
# sns.heatmap(df_train.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [9]:
# plt.figure(figsize=(30,30))
# sns.heatmap(pd.concat([df_train,output_col],axis=1).loc[:,list(pd.concat([df_train,output_col],axis=1).select_dtypes(include= np.number).columns)].corr(method='spearman'),annot=True,cmap="RdYlGn")
# plt.show()

In [10]:
# fig, axes = plt.subplots(nrows = 12, ncols = 3)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(30,30)
# cols=list(df_train.select_dtypes(include= np.number).columns)
# for ax, col in zip(axes, cols):
#     print()
#     sns.boxplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [11]:
# fig, axes = plt.subplots(nrows = 12, ncols = 3)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(30,30)
# cols=list(df_train.select_dtypes(include= np.number).columns)
# for ax, col in zip(axes, cols):
#     print()
#     sns.distplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [12]:

def column_with_most_reduntant_null_values(threshold,df_train:pd.DataFrame):
    null_cols=[]
    for col in df_train.columns:
        if df_train[col].isnull().sum()>=(threshold*len(df_train)):
            null_cols.append(col)
    df_train.drop(null_cols,axis=1,inplace=True)

In [13]:
column_with_most_reduntant_null_values(threshold=0.5,df_train=df_train)

In [14]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,61,0,0,0,0,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,42,0,0,0,0,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,35,272,0,0,0,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,84,0,0,0,0,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,40,0,0,0,0,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,60,0,0,0,0,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,112,0,0,0,0,4,2010,WD,Normal


In [15]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [16]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

In [17]:
most_correlated=dict()
for col1 in continuous_cols:
    for col2 in continuous_cols:
        if col1!=col2 and abs(df_train[col1].corr(df_train[col2],method='spearman'))>=0.80:
            if (col1 not in most_correlated) and (col2 not in most_correlated):
                most_correlated[col1]=col2

In [18]:
df_train.drop(columns=list(most_correlated),axis=1,inplace=True)

In [19]:
# Filling Null Values

def fill_null_values(for_numerical_cols:str,for_categorical_cols:str,df_train):
    for cols in df_train.columns:
        if df_train[cols].dtype=="int64" or df_train[cols].dtype=="float64":
            if for_numerical_cols=="median":
                median_=df_train[cols].median()
                df_train[cols].fillna(median_,inplace=True)
        elif df_train[cols].dtype=="object":
            if for_categorical_cols=="most_frequent":
                d=list(df_train[cols].value_counts().index) # most frequent
                df_train[cols].fillna(d[0],inplace=True)


In [20]:
fill_null_values(for_numerical_cols="median",for_categorical_cols="most_frequent",df_train=df_train)

In [21]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>(0):

        print(col,df[col].isnull().sum(),df_train[col].dtypes)


In [22]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,61,0,0,0,0,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,42,0,0,0,0,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,35,272,0,0,0,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,84,0,0,0,0,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,40,0,0,0,0,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,60,0,0,0,0,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,112,0,0,0,0,4,2010,WD,Normal


In [23]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [24]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

In [25]:
df_train=pd.concat([output_col,df_train],axis=1) # since rows will be deleted here


In [26]:

# Outlier delete

for col in continuous_cols:
    q1=df_train[col].quantile(0.25)
    q3=df_train[col].quantile(0.75)
    iqr=q3-q1
    l=q1-1.5*iqr
    h=q3+1.5*iqr
    print("[",l,",",h,"]")
    df_train = df_train[(df_train[col] <= h)] 
    df_train = df_train[(df_train[col] >=l)] 


[ -55.0 , 145.0 ]
[ 30.0 , 110.0 ]
[ 2440.0 , 16936.0 ]
[ 2.0 , 10.0 ]
[ 3.5 , 7.5 ]
[ 1909.0 , 2061.0 ]
[ -255.75 , 426.25 ]
[ -1060.5 , 1767.5 ]
[ 0.0 , 0.0 ]
[ -579.875 , 1725.125 ]
[ 199.0 , 2037.0 ]
[ -1092.375 , 1820.625 ]
[ 0.0 , 0.0 ]
[ -1.5 , 2.5 ]
[ 0.0 , 0.0 ]
[ -0.5 , 3.5 ]
[ -1.5 , 2.5 ]
[ 0.5 , 4.5 ]
[ 1.0 , 1.0 ]
[ 2.0 , 10.0 ]
[ -1.5 , 2.5 ]
[ 1903.0 , 2063.0 ]
[ -19.0 , 933.0 ]
[ -239.625 , 399.375 ]
[ -97.5 , 162.5 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.5 , 12.5 ]
[ 2004.0 , 2012.0 ]


In [27]:
output_col=df_train['SalePrice']
df_train.drop("SalePrice",axis=1,inplace=True)

In [28]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,61,0,0,0,0,0,2,2008,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,42,0,0,0,0,0,9,2008,WD,Normal
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,84,0,0,0,0,0,12,2008,WD,Normal
6,20,RL,75.0,10084,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,57,0,0,0,0,0,8,2007,WD,Normal
10,20,RL,70.0,11200,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,2,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,20,RL,63.0,8500,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,60,0,0,0,0,0,11,2007,WD,Normal
1448,50,RL,70.0,11767,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,24,0,0,0,0,0,5,2007,WD,Normal
1451,20,RL,78.0,9262,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,36,0,0,0,0,0,5,2009,New,Partial
1454,20,FV,62.0,7500,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,113,0,0,0,0,0,10,2009,WD,Normal


In [29]:


def encoding_categorical_cols(df_train:pd.DataFrame,categorical_columns:list,strategy:str):
    df_categorical=pd.DataFrame(df_train.loc[:,categorical_columns])
    if strategy=="labelencoder":
        for col in list(df_categorical.columns):
            cat=[]
            d={}

            cat.append([df_categorical[col].value_counts().index,len(df_categorical[col].value_counts().index)])

            for j in range(int(cat[0][1])):
                d[str(cat[0][0][j])]=j
            df_categorical[col]=df_categorical[col].map(d)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        
        df_train=pd.concat([df_train,df_categorical],axis=1)
        
    if strategy=="onehotencoder":
        one_hot_encoded=pd.get_dummies(df_categorical)
        # print(one_hot_encoded)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        df_train=pd.concat([df_train,one_hot_encoded],axis=1)
    return df_train
        

    
    


One hot encoding since hyperparameter tuning does not have its effects

In [30]:
df_train=encoding_categorical_cols(df_train=df_train,categorical_columns=categorical_cols,strategy="onehotencoder")

In [31]:
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,196.0,706,0,150,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2002,162.0,486,0,434,...,0,0,0,1,0,0,0,0,1,0
4,60,84.0,14260,8,5,2000,350.0,655,0,490,...,0,0,0,1,0,0,0,0,1,0
6,20,75.0,10084,8,5,2005,186.0,1369,0,317,...,0,0,0,1,0,0,0,0,1,0
10,20,70.0,11200,5,5,1965,0.0,906,0,134,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,20,63.0,8500,7,5,2004,106.0,0,0,1422,...,0,0,0,1,0,0,0,0,1,0
1448,50,70.0,11767,4,7,2000,0.0,0,0,560,...,0,0,0,1,0,0,0,0,1,0
1451,20,78.0,9262,8,5,2009,194.0,0,0,1573,...,0,1,0,0,0,0,0,0,0,1
1454,20,62.0,7500,7,5,2005,0.0,410,0,811,...,0,0,0,1,0,0,0,0,1,0


In [32]:
### SCALING

In [33]:
scaler=scaler=MinMaxScaler()
df_train=pd.DataFrame(scaler.fit_transform(df_train),columns=df_train.columns,)
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.4,0.4375,0.400706,0.625,0.333333,0.883333,0.463357,0.436072,0.0,0.087771,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.4,0.4750,0.602391,0.625,0.333333,0.866667,0.382979,0.300185,0.0,0.253950,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.4,0.6750,0.819203,0.750,0.333333,0.833333,0.827423,0.404571,0.0,0.286717,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.5625,0.518404,0.750,0.333333,0.916667,0.439716,0.845584,0.0,0.185489,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.5000,0.598790,0.375,0.333333,0.250000,0.000000,0.559605,0.0,0.078408,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,0.0,0.4125,0.404307,0.625,0.333333,0.900000,0.250591,0.000000,0.0,0.832066,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
572,0.3,0.5000,0.639631,0.250,1.000000,0.833333,0.000000,0.000000,0.0,0.327677,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
573,0.0,0.6000,0.459195,0.750,0.333333,0.983333,0.458629,0.000000,0.0,0.920421,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
574,0.0,0.4000,0.332277,0.625,0.333333,0.916667,0.000000,0.253243,0.0,0.474547,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [34]:
from sklearn.metrics import r2_score

In [35]:
X=df_train
Y=output_col

In [36]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=None,shuffle=True)


In [37]:
final_accuracies={}

Atlast we will be doing manual hyperparameter tuning. Firstly we wiil do Random SearchCV since it gives us a region in the form of parameters where our accuracy could we very high,this reduces our search space. Then using those parameters we will use Grid Search CV to get best result using combinations of those parameters.

In [38]:
def hyperparameter_tuning(algo:str,param_grid:dict,n_iter:int,cv:int,verbose:int,random_state:int,X_train,Y_train,X_test,Y_test,evaluation_metric:str):
    model=eval(algo)
    evaluation_metric=eval(evaluation_metric)
    
    model_random_cv=RandomizedSearchCV(estimator=model,param_distributions=param_grid,n_iter=n_iter,cv=cv,verbose=verbose,random_state=random_state,n_jobs=-1)
    model_random_cv.fit(X_train,Y_train)
    best_params_=model_random_cv.best_params_
    print(f"{model} RandomCV Best Params :",best_params_)
    best_random_grid=model_random_cv.best_estimator_
    y_pred=best_random_grid.predict(X_test)
    
    score=evaluation_metric(Y_test,y_pred)
    print(f"{model} RandomCV Score:",score)
    
    
    
    param_grid=defaultdict(list)
    for val in best_params_:
        param_grid[val].append(best_params_[val])
    
    
    model_for_gcv=eval(algo)
    
    grid_search=GridSearchCV(estimator=model_for_gcv,param_grid=param_grid,cv=cv,n_jobs=-1,verbose=verbose)
    grid_search.fit(X_train,Y_train)
    best_grid2=grid_search.best_estimator_
    y_pred2=best_grid2.predict(X_test)
    score_1=evaluation_metric(Y_test,y_pred2)
    print("Final score is ",score_1)
    final_accuracies[algo]=score_1
    
    
   

In [39]:
## On Linear Regresion


In [40]:
# On Linear Regresion
fit_intercept = ["True","False"]
copy_X =["True","False"] 
positive=["False","True"]

random_grid = {'fit_intercept': fit_intercept,
               'positive': positive,
               'copy_X': copy_X}
print(random_grid)

{'fit_intercept': ['True', 'False'], 'positive': ['False', 'True'], 'copy_X': ['True', 'False']}


In [41]:
hyperparameter_tuning(algo="LinearRegression()",param_grid=random_grid,n_iter=100,cv=5,verbose=3,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\model_selection\_search.py:296: UserWarning: The total space of parameters 8 is smaller than n_iter=100. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


Fitting 5 folds for each of 8 candidates, totalling 40 fits
LinearRegression() RandomCV Best Params : {'positive': 'False', 'fit_intercept': 'True', 'copy_X': 'True'}
LinearRegression() RandomCV Score: 0.9025916718957617
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Final score is  0.9025916718957617


In [42]:
# RandomForestRegressor

In [43]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [2,3,4, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['squared_error','absolute_error','friedman_mse','poisson']}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 3, 4, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']}


In [44]:
hyperparameter_tuning(algo="RandomForestRegressor()",param_grid=random_grid,n_iter=10,cv=5,verbose=3,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
RandomForestRegressor() RandomCV Best Params : {'n_estimators': 200, 'min_samples_split': 14, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 890, 'criterion': 'friedman_mse'}
RandomForestRegressor() RandomCV Score: 0.8376378316567984
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Final score is  0.8417155960544276


In [236]:
# 3. SVR

In [47]:
kernel = ["linear", "poly", "sigmoid","poly"]
degree = [int(x) for x in np.linspace(start = 1, stop = 1000, num = 100)]
gamma= ["scale", "auto"]

tol=[1e-2]
max_iter=[100]


random_grid = {'kernel': kernel,
               'degree':degree,
               'max_iter':max_iter,
               'tol':tol,
               'gamma':gamma
               
               
               }
print(random_grid)

{'kernel': ['linear', 'poly', 'sigmoid', 'poly'], 'degree': [1, 11, 21, 31, 41, 51, 61, 71, 81, 91, 101, 112, 122, 132, 142, 152, 162, 172, 182, 192, 202, 212, 223, 233, 243, 253, 263, 273, 283, 293, 303, 313, 323, 334, 344, 354, 364, 374, 384, 394, 404, 414, 424, 434, 445, 455, 465, 475, 485, 495, 505, 515, 525, 535, 545, 556, 566, 576, 586, 596, 606, 616, 626, 636, 646, 656, 667, 677, 687, 697, 707, 717, 727, 737, 747, 757, 767, 778, 788, 798, 808, 818, 828, 838, 848, 858, 868, 878, 889, 899, 909, 919, 929, 939, 949, 959, 969, 979, 989, 1000], 'max_iter': [100], 'tol': [0.01], 'gamma': ['scale', 'auto']}


In [48]:
hyperparameter_tuning(algo="SVR()",param_grid=random_grid,n_iter=10,cv=5,verbose=0,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

SVR() RandomCV Best Params : {'tol': 0.01, 'max_iter': 100, 'kernel': 'poly', 'gamma': 'auto', 'degree': 667}
SVR() RandomCV Score: -0.008718582334627412
Final score is  -0.008718582334627412


C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\model_selection\_search.py:972: UserWarning: One or more of the test scores are non-finite: [-0.05990195         nan -0.02193293 -0.02048921 -0.1469418  -0.02048921
 -0.1469418  -0.1469418  -0.05990195 -0.05990195]
  category=UserWarning,
C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


In [244]:
# 4 KNN Regressor

In [49]:
weights=["uniform", "distance"]
n_neighbors = [int(x) for x in np.linspace(start = 5, stop = 350, num = 50)]
algorithm= ["auto", "ball_tree","kd_tree","brute"]
leaf_size=[int(x) for x in np.linspace(start = 5, stop = 20, num = 10)]
p = [1,2]


random_grid = {'weights': weights,
               'n_neighbors': n_neighbors,
               'algorithm': algorithm,
               'leaf_size':leaf_size,
               'p':p
               }
print(random_grid)

{'weights': ['uniform', 'distance'], 'n_neighbors': [5, 12, 19, 26, 33, 40, 47, 54, 61, 68, 75, 82, 89, 96, 103, 110, 117, 124, 131, 138, 145, 152, 159, 166, 173, 181, 188, 195, 202, 209, 216, 223, 230, 237, 244, 251, 258, 265, 272, 279, 286, 293, 300, 307, 314, 321, 328, 335, 342, 350], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 'leaf_size': [5, 6, 8, 10, 11, 13, 15, 16, 18, 20], 'p': [1, 2]}


In [50]:
hyperparameter_tuning(algo="KNeighborsRegressor()",param_grid=random_grid,n_iter=10,cv=5,verbose=2,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
KNeighborsRegressor() RandomCV Best Params : {'weights': 'distance', 'p': 2, 'n_neighbors': 5, 'leaf_size': 18, 'algorithm': 'brute'}
KNeighborsRegressor() RandomCV Score: 0.7547009547817669
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Final score is  0.7547009547817669


In [251]:
## 5 DecisionTreeRegressor

In [51]:
splitter=["best", "random"]
# Number of features to consider at every split
max_features = ['sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,100)]
# Minimum number of samples required to split a node
min_samples_split = [2,3,4, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1,2, 4,6,8]
# Create the random grid
random_grid = {'splitter':splitter,
               'ccp_alpha':[1,0],
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['squared_error','absolute_error','friedman_mse','poisson']}
print(random_grid)

{'splitter': ['best', 'random'], 'ccp_alpha': [1, 0], 'max_features': ['sqrt', 'log2'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990, 1000], 'min_samples_split': [2, 3, 4, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']}


In [52]:
hyperparameter_tuning(algo="DecisionTreeRegressor()",param_grid=random_grid,n_iter=10,cv=5,verbose=2,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
DecisionTreeRegressor() RandomCV Best Params : {'splitter': 'best', 'min_samples_split': 14, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'max_depth': 580, 'criterion': 'squared_error', 'ccp_alpha': 0}
DecisionTreeRegressor() RandomCV Score: 0.6950225584543751
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Final score is  0.7496116878640268


In [257]:
## 6 SGDRegressor

In [57]:
loss = ['squared_error','huber','epsilon_insensitive','squared_epsilon_insensitive']
penalty=['l2','l1','elasticnet','None']
alpha = [float(x) for x in np.linspace(0, 0.001,50)]
l1_ratio=[float(x) for x in np.linspace(0, 0.5,50)]
max_iter=[int(x) for x in np.linspace(8000, 20000,100)]
tol=[float(x) for x in np.linspace(0.01, 0.001,100)]
learning_rate=['constant','optimal','invscaling','adaptive']
eta0=[float(x) for x in np.linspace(0.1, 0.05,100)]
early_stopping = ['True','False']
validation_fraction=[float(x) for x in np.linspace(0.1, 0.9,50)]
n_iter_no_change = [1,2, 4,6,8,5,10]
average=['bool','int']
random_grid = {
               'loss': loss,
               'penalty': penalty,
               'alpha': alpha,
               'l1_ratio': l1_ratio,
               'max_iter':max_iter,
               'tol':tol,
               'learning_rate':learning_rate,
               'eta0':eta0,
               'validation_fraction':validation_fraction,
               'n_iter_no_change':n_iter_no_change
                }
print(random_grid)

{'loss': ['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'], 'penalty': ['l2', 'l1', 'elasticnet', 'None'], 'alpha': [0.0, 2.0408163265306123e-05, 4.0816326530612245e-05, 6.122448979591836e-05, 8.163265306122449e-05, 0.00010204081632653062, 0.00012244897959183673, 0.00014285714285714287, 0.00016326530612244898, 0.0001836734693877551, 0.00020408163265306123, 0.00022448979591836734, 0.00024489795918367346, 0.0002653061224489796, 0.00028571428571428574, 0.0003061224489795918, 0.00032653061224489796, 0.0003469387755102041, 0.0003673469387755102, 0.0003877551020408163, 0.00040816326530612246, 0.00042857142857142855, 0.0004489795918367347, 0.00046938775510204083, 0.0004897959183673469, 0.0005102040816326531, 0.0005306122448979592, 0.0005510204081632653, 0.0005714285714285715, 0.0005918367346938776, 0.0006122448979591836, 0.0006326530612244898, 0.0006530612244897959, 0.000673469387755102, 0.0006938775510204082, 0.0007142857142857143, 0.0007346938775510204, 0.0007

In [58]:
hyperparameter_tuning(algo="SGDRegressor()",param_grid=random_grid,n_iter=10,cv=5,verbose=2,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

OverflowError: Python int too large to convert to C long

In [263]:
accuracies

{'LinearRegression()': -1.3643518980430038e+23,
 'RandomForestRegressor()': 0.8201522653525275,
 'SVR()': -7.943707625401863e-05,
 'KNeighborsRegressor()': 0.7073273687494936,
 'DecisionTreeRegressor()': 0.6699070971484998,
 'SGDRegressor()': 0.7848215372436905}

In [264]:
# 7 ElasticNet

In [59]:
alpha = [float(x) for x in np.linspace(0, 0.50,500)]
positive=['True','False']
l1_ratio=[float(x) for x in np.linspace(0, 0.5,100)]
max_iter=[int(x) for x in np.linspace(2000, 10000,200)]
tol=[float(x) for x in np.linspace(0.01, 0.001,200)]
warm_start = ['True','False']
selection=['cyclic','random']
copy_X=['True','False']

random_grid = {'copy_X':copy_X,
               'alpha':alpha,
               'l1_ratio': l1_ratio,
               'max_iter':max_iter,
               'tol':tol,
                'warm_start':warm_start,
                'selection':selection
               
               
                }
print(random_grid)

{'copy_X': ['True', 'False'], 'alpha': [0.0, 0.001002004008016032, 0.002004008016032064, 0.0030060120240480957, 0.004008016032064128, 0.00501002004008016, 0.0060120240480961915, 0.007014028056112224, 0.008016032064128256, 0.009018036072144287, 0.01002004008016032, 0.011022044088176352, 0.012024048096192383, 0.013026052104208416, 0.014028056112224447, 0.01503006012024048, 0.01603206412825651, 0.017034068136272545, 0.018036072144288574, 0.019038076152304607, 0.02004008016032064, 0.02104208416833667, 0.022044088176352703, 0.023046092184368736, 0.024048096192384766, 0.0250501002004008, 0.026052104208416832, 0.027054108216432865, 0.028056112224448895, 0.029058116232464928, 0.03006012024048096, 0.03106212424849699, 0.03206412825651302, 0.033066132264529056, 0.03406813627254509, 0.03507014028056112, 0.03607214428857715, 0.03707414829659318, 0.038076152304609215, 0.03907815631262525, 0.04008016032064128, 0.041082164328657314, 0.04208416833667334, 0.04308617234468937, 0.044088176352705406, 0.04

In [60]:
hyperparameter_tuning(algo="ElasticNet()",param_grid=random_grid,n_iter=10,cv=5,verbose=2,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

OverflowError: Python int too large to convert to C long

In [270]:
# 8 Lasso

In [61]:
max_iter=[int(x) for x in np.linspace(2000, 10000,500)]
tol=[float(x) for x in np.linspace(0.01, 0.001,300)]
warm_start = ['True','False']
selection=['cyclic','random']
alpha = [float(x) for x in np.linspace(0,0.50,500)]
# positive=['True','False']
copy_X=['True','False']
random_grid = {
               'alpha':alpha,
              
               'max_iter':max_iter,
               'tol':tol,
                'positive':positive,
                'copy_X':copy_X,
                'warm_start':warm_start,
                'selection':selection
               
               
                }
print(random_grid)


{'alpha': [0.0, 0.001002004008016032, 0.002004008016032064, 0.0030060120240480957, 0.004008016032064128, 0.00501002004008016, 0.0060120240480961915, 0.007014028056112224, 0.008016032064128256, 0.009018036072144287, 0.01002004008016032, 0.011022044088176352, 0.012024048096192383, 0.013026052104208416, 0.014028056112224447, 0.01503006012024048, 0.01603206412825651, 0.017034068136272545, 0.018036072144288574, 0.019038076152304607, 0.02004008016032064, 0.02104208416833667, 0.022044088176352703, 0.023046092184368736, 0.024048096192384766, 0.0250501002004008, 0.026052104208416832, 0.027054108216432865, 0.028056112224448895, 0.029058116232464928, 0.03006012024048096, 0.03106212424849699, 0.03206412825651302, 0.033066132264529056, 0.03406813627254509, 0.03507014028056112, 0.03607214428857715, 0.03707414829659318, 0.038076152304609215, 0.03907815631262525, 0.04008016032064128, 0.041082164328657314, 0.04208416833667334, 0.04308617234468937, 0.044088176352705406, 0.04509018036072144, 0.0460921843

In [273]:
hyperparameter_tuning(algo="Lasso()",param_grid=random_grid,n_iter=10,cv=5,verbose=2,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 4/5] END copy_X=True, fit_intercept=True, positive=False;, score=0.722 total time=   0.1s
[CV 1/5] END copy_X=True, fit_intercept=False, positive=False;, score=0.686 total time=   0.1s
[CV 2/5] END copy_X=True, fit_intercept=False, positive=True;, score=0.559 total time=   0.1s
[CV 2/5] END copy_X=False, fit_intercept=True, positive=False;, score=0.559 total time=   0.1s
[CV 3/5] END copy_X=False, fit_intercept=True, positive=True;, score=-1491335583445575312343040.000 total time=   0.1s
[CV 4/5] END copy_X=False, fit_intercept=False, positive=False;, score=0.722 total time=   0.1s
[CV 5/5] END copy_X=False, fit_intercept=False, positive=True;, score=0.772 total time=   0.1s
[CV 1/5] END criterion=poisson, max_depth=1000, max_features=sqrt, min_samples_leaf=8, min_samples_split=3, n_estimators=1000;, score=0.830 total time=   1.8s
[CV 3/5] END criterion=friedman_mse, max_depth=670, max_features=sqrt, min_samples_leaf=6, m

In [278]:
# 9 Ridge

In [62]:
max_iter=[int(x) for x in np.linspace(2000, 10000,200)]
tol=[float(x) for x in np.linspace(0.01, 0.001,200)]
alpha = [float(x) for x in np.linspace(0.1,0.50,400)]
positive=['False']
copy_X=['True','False']
solver=["auto"]

random_grid = {
               'alpha':alpha,
              
               'max_iter':max_iter,
               'tol':tol,
                'positive':positive,
                'copy_X':copy_X,
                'solver':solver
               
               
                }
print(random_grid)

{'alpha': [0.1, 0.10100250626566416, 0.10200501253132832, 0.10300751879699249, 0.10401002506265665, 0.10501253132832081, 0.10601503759398496, 0.10701754385964912, 0.1080200501253133, 0.10902255639097745, 0.11002506265664161, 0.11102756892230577, 0.11203007518796992, 0.1130325814536341, 0.11403508771929825, 0.11503759398496241, 0.11604010025062657, 0.11704260651629073, 0.1180451127819549, 0.11904761904761905, 0.12005012531328321, 0.12105263157894737, 0.12205513784461153, 0.1230576441102757, 0.12406015037593986, 0.12506265664160401, 0.12606516290726819, 0.12706766917293233, 0.1280701754385965, 0.12907268170426067, 0.13007518796992482, 0.131077694235589, 0.13208020050125313, 0.1330827067669173, 0.13408521303258147, 0.13508771929824562, 0.1360902255639098, 0.13709273182957393, 0.1380952380952381, 0.13909774436090228, 0.14010025062656642, 0.1411027568922306, 0.14210526315789473, 0.1431077694235589, 0.14411027568922308, 0.14511278195488722, 0.1461152882205514, 0.14711779448621554, 0.14812030

In [63]:
hyperparameter_tuning(algo="Ridge()",param_grid=random_grid,n_iter=10,cv=5,verbose=2,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Ridge() RandomCV Best Params : {'tol': 0.0015879396984924618, 'solver': 'auto', 'positive': 'False', 'max_iter': 7226, 'copy_X': 'True', 'alpha': 0.2824561403508772}
Ridge() RandomCV Score: 0.9136787376017803
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Final score is  0.9136787376017803


In [64]:
accuracies

NameError: name 'accuracies' is not defined